In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions/text.csv


In [19]:
#import torch as nn
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoModel, TrainingArguments,Trainer
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F

# Explore the Data

In [5]:
df = pd.read_csv("/kaggle/input/emotions/text.csv", index_col=[0], nrows=5000) # chunksize=10000
df

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
4995,i long every moment to feel your gentle touch,2
4996,im just not feeling that needy or messed up an...,0
4997,i hate him and the feeling is pretty mutual i ...,3
4998,i can guarantee you that if i saw these jerks ...,4


In [6]:
df.describe()

,label
count,5000.000000
mean,1.576000
std,1.504353
min,0.000000
25%,0.000000
50%,1.000000
75%,3.000000
max,5.000000


In [7]:
df.describe(include="object")

,text
count,5000
unique,4999
top,i feel like if i continue this i will turn int...
freq,2


In [8]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 5000
})

In [9]:
ds = ds.rename_column("__index_level_0__", "id")
ds = ds.rename_column("label", "labels") # huggingface trainer expects label column to be named labels

# Finetune with Distilbert

In [10]:
model_nm = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(model_nm)
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=6)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
tokenized_ds

Dataset({
    features: ['text', 'labels', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})

In [13]:
split_ds = tokenized_ds.train_test_split(0.25)
split_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3750
    })
    test: Dataset({
        features: ['text', 'labels', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1250
    })
})

# Finetuning

In [108]:
# def corr(x,y): return np.corrcoef(x,y)[0][1]
# def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [14]:
# Define the accuracy metric
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

In [15]:
bs = 64
epochs = 4
lr = 8e-5

In [16]:
args = TrainingArguments(
    'outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", logging_strategy="steps", logging_steps=10,  # Logs every 10 steps
    per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none'
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(model, args, train_dataset=split_ds['train'], eval_dataset=split_ds['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.237500,1.114002,0.579200
2,0.769200,0.670715,0.771200
3,0.334900,0.392154,0.888000
4,0.237000,0.377464,0.885600


TrainOutput(global_step=236, training_loss=0.7538540454234107, metrics={'train_runtime': 68.3392, 'train_samples_per_second': 219.493, 'train_steps_per_second': 3.453, 'total_flos': 219639947450400.0, 'train_loss': 0.7538540454234107, 'epoch': 4.0})

# Test Model

In [20]:
input_text = "i guess that feeling is what im really getting all nostalgic about"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Convert the tokenized input to a Dataset
eval_ds = Dataset.from_dict({
    
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask']
})
eval_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1
})

In [21]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[-1.70703125,  0.65332031,  3.1875    , -1.92285156, -1.5625    ,
         1.70507812]])

In [22]:
# Apply softmax to convert logits to probabilities (optional, for interpretation)
probs = F.softmax(torch.tensor(preds), dim=-1)

# Get the predicted class (index of the largest logit)
predicted_class = torch.argmax(probs, dim=1).item()

# Print the predicted class
print(f"Predicted class: {predicted_class}")

Predicted class: 2
